In [1]:
cd .. 

/home/mehdi2277/Documents/HarveyMuddWork/Neural_Nets_Research/neural_nets_research


In [2]:
# EDIT THESE FIELDS TO DETERMINE HOW MANY TRAINING RUNS WE DO FOR WHAT FILE
training_rounds = 5
decoder_type = "tree"
save_file_name = "t2t"
which_cuda = 0
print("done")

done


In [3]:
import torch.optim as optim
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
import os

from neural_nets_library import training
from tree_to_sequence.tree_encoder import TreeEncoder
from tree_to_sequence.tree_decoder import TreeDecoder
from tree_to_sequence.tree_to_sequence_attention import TreeToSequenceAttention
from tree_to_sequence.grammar_tree_decoder import GrammarTreeDecoder
from tree_to_sequence.multilayer_lstm_cell import MultilayerLSTMCell
from tree_to_sequence.program_datasets import *
from tree_to_sequence.translating_trees import ( parent_to_category_FOR, category_to_child_FOR,
                                                 category_to_child_LAMBDA, 
                                                 parent_to_category_LAMBDA, ForGrammar, For,
                                                 LambdaGrammar, Lambda)
from tree_to_sequence.tree_to_tree_attention import TreeToTreeAttention
from functools import partial

In [4]:
torch.cuda.set_device(which_cuda)

In [5]:
# Counts the number of matches between the prediction and target.
def count_matches(prediction, target):
    matches = 0
    if int(prediction.value) == int(target.value):
        matches += 1
    for i in range(min(len(target.children), len(prediction.children))):
        matches += count_matches(prediction.children[i], target.children[i])
    return matches

# Program accuracy (1 if completely correct, 0 otherwise)
def program_accuracy(prediction, target):
    if decoder_type == "sequence":
        target_list = [int(x) for x in list(target.data[:-1])]
        return 1 if target_list == prediction else 0
    if prediction.size() == count_matches(prediction, target) and \
       prediction.size() == target.size():
        return 1
    else:
        return 0

# Calculate validation accuracy (this could either be program or token accuracy)
def validation_criterion(prediction, target):
    return program_accuracy(prediction, target)

def reset_all_parameters_uniform(model, stdev):
    for param in model.parameters():
        nn.init.uniform_(param, -stdev, stdev)

In [13]:
num_vars = 10
num_ints = 11
one_hot = False
binarize = True
eos_token = (decoder_type != "grammar")
long_base_case = True
input_as_seq = False
output_as_seq = (decoder_type == "sequence")
use_cuda = True
binarize_output = (decoder_type == "tree")

In [14]:
# Make dataset
dset_train = ForLambdaDataset("ANC/training_For.json",
                                   binarize_input=binarize, binarize_output=binarize_output, 
                                   eos_token=eos_token, one_hot=one_hot,
                                   num_ints=num_ints, num_vars=num_vars,
                                   long_base_case=long_base_case, 
                                   input_as_seq=input_as_seq, 
                                   output_as_seq=output_as_seq)

dset_val = ForLambdaDataset("ANC/validation_For.json",
                                   binarize_input=binarize, binarize_output=binarize_output, 
                                   eos_token=eos_token, one_hot=one_hot,
                                   num_ints=num_ints, num_vars=num_vars,
                                   long_base_case=long_base_case, 
                                   input_as_seq=input_as_seq, 
                                   output_as_seq=output_as_seq)

dset_test = ForLambdaDataset("ANC/test_For.json",
                                   binarize_input=binarize, binarize_output=binarize_output, 
                                   eos_token=eos_token, one_hot=one_hot,
                                   num_ints=num_ints, num_vars=num_vars,
                                   long_base_case=long_base_case, 
                                   input_as_seq=input_as_seq, 
                                   output_as_seq=output_as_seq)

In [15]:
max_size = int(max([x[1].size() for x in dset_train] + [x[1].size() for x in dset_val] + [x[1].size() for x in dset_test]))
embedding_size = 256
hidden_size = 256
num_layers = 1
alignment_size = 50
align_type = 1
encoder_input_size = num_vars + num_ints + len(for_ops)
nclass = num_vars + num_ints + len(lambda_ops)
plot_every = 100
num_categories = len(LambdaGrammar)
num_possible_parents = len(Lambda)
save_every=5000
max_num_children = 2 if binarize else 4

In [16]:
def save_plots():
    # Save plots
    plt.plot([x * plot_every for x in range(len(train_plot_losses))], train_plot_losses)
    plt.plot([x * plot_every for x in range(len(val_plot_losses))], val_plot_losses)
    plt.title("Training Loss")
    plt.xlabel('Training Examples')
    plt.ylabel('Cross-Entropy Loss')
    plt.legend(("Train", "Validation"))
    plt.savefig(save_folder + "/" + save_file + "_train_plot.png")
    plt.close()

    plt.plot([x * plot_every for x in range(len(train_plot_accuracies))], train_plot_accuracies)
    plt.plot([x * plot_every for x in range(len(val_plot_accuracies))], val_plot_accuracies)
    plt.title("Training Program Accuracy")
    plt.xlabel('Training Examples')
    plt.ylabel('Percent Accurate Programs')
    plt.legend(("Train", "Validation"))
    plt.savefig(save_folder + "/" + save_file + "_accuracy_plot.png")

In [10]:
def save_test_accuracies():
    with open(save_folder + "/" + save_file + "_test.txt", "a") as file:
        for val in test_accuracies:
            file.write(str(val) + ",")

In [17]:
def make_model():
    encoder = TreeEncoder(encoder_input_size, hidden_size, num_layers, [1, 2, 3, 4, 5], attention=True, one_hot=one_hot)

    if decoder_type == "grammar":
        decoder = GrammarTreeDecoder(embedding_size, hidden_size, num_categories, 
                             num_possible_parents, 
                             partial(parent_to_category_LAMBDA, num_vars, num_ints), 
                             partial(category_to_child_LAMBDA, num_vars, num_ints), 
                             share_linear=True, share_lstm_cell=True, 
                             num_ints_vars=num_ints + num_vars)
        
        program_model = TreeToTreeAttention(encoder, decoder, 
                                        hidden_size, embedding_size, 
                                        nclass=nclass, root_value=nclass,
                                        alignment_size=alignment_size, 
                                        align_type=align_type, max_size=max_size)
        
    elif decoder_type == "tree":
        decoder = TreeDecoder(embedding_size, hidden_size, max_num_children, nclass=nclass)
        
        program_model = TreeToTreeAttention(encoder, decoder, 
                                        hidden_size, embedding_size, 
                                        nclass=nclass, root_value=nclass,
                                        alignment_size=alignment_size, 
                                        align_type=align_type, max_size=max_size)
        
    elif decoder_type == "sequence":
        decoder = MultilayerLSTMCell(embedding_size + hidden_size, hidden_size, num_layers)
        
        program_model = TreeToSequenceAttention(encoder, decoder, hidden_size, nclass, 
                                                embedding_size, alignment_size=alignment_size, 
                                                align_type=align_type)
        
    if use_cuda:
        program_model = program_model.cuda()
        
    reset_all_parameters_uniform(program_model, 0.1)
    encoder.initialize_forget_bias(3)
    decoder.initialize_forget_bias(3)
    
    return program_model

In [ ]:
test_accuracies = []
for i in range(training_rounds):
    program_model = make_model()
    save_file = save_file_name + "_" + str(i)
    save_folder = "test_various_models"
    
    # Optimizer
    optimizer = optim.Adam(program_model.parameters(), lr=0.005)
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=500, factor=0.8)

    # Train
    os.system("mkdir "+ save_folder)
    best_model, train_plot_losses, train_plot_accuracies, val_plot_losses, val_plot_accuracies = training.train_model_tree_to_tree(
                                    program_model, 
                                    dset_train, 
                                    optimizer, 
                                    lr_scheduler=lr_scheduler, 
                                    num_epochs=5, plot_every=plot_every,
                                    batch_size=100, 
                                    print_every=200, 
                                    validation_dset = dset_val,                                                          
                                    validation_criterion=validation_criterion,
                                    use_cuda=use_cuda, 
                                    deep_copy_desired=False,
                                    plateau_lr=True,
                                    save_file=save_file,
                                    save_folder=save_folder,
                                    save_every=save_every)
    
    # Test
    test_accuracy = training.test_model_tree_to_tree(best_model, dset_test, validation_criterion) #TODO: change where best_model is saved
    test_accuracies.append(test_accuracy)
    save_plots()
    
save_test_accuracies()
        
    

Epoch 0/4
----------
Epoch Number: 0, Batch Number: 200, Training Loss: 240.8770
Time so far is 1m 11s
Epoch Number: 0, Batch Number: 200, Validation Metric: 0.0000
Example output:
Epoch Number: 0, Batch Number: 400, Training Loss: 156.0125
Time so far is 1m 44s
Epoch Number: 0, Batch Number: 400, Validation Metric: 0.0000
Example output:
Epoch Number: 0, Batch Number: 600, Training Loss: 123.8027
Time so far is 2m 17s
Epoch Number: 0, Batch Number: 600, Validation Metric: 0.0000
Example output:
Epoch Number: 0, Batch Number: 800, Training Loss: 112.6162
Time so far is 2m 49s
Epoch Number: 0, Batch Number: 800, Validation Metric: 0.0000
Example output:
Epoch Number: 0, Batch Number: 1000, Training Loss: 108.0209
Time so far is 3m 19s
Epoch Number: 0, Batch Number: 1000, Validation Metric: 0.0000
Example output:
Epoch Number: 0, Batch Number: 1200, Training Loss: 104.0252
Time so far is 3m 51s
Epoch Number: 0, Batch Number: 1200, Validation Metric: 0.0000
Example output:
Epoch Number: 0

Epoch Number: 0, Batch Number: 10400, Training Loss: 16.7072
Time so far is 34m 14s
Epoch Number: 0, Batch Number: 10400, Validation Metric: 0.1550
Example output:
Epoch Number: 0, Batch Number: 10600, Training Loss: 18.7833
Time so far is 34m 56s
Epoch Number: 0, Batch Number: 10600, Validation Metric: 0.1300
Example output:
Epoch Number: 0, Batch Number: 10800, Training Loss: 17.2068
Time so far is 35m 38s
Epoch Number: 0, Batch Number: 10800, Validation Metric: 0.1550
Example output:
Epoch Number: 0, Batch Number: 11000, Training Loss: 16.2868
Time so far is 36m 17s
Epoch Number: 0, Batch Number: 11000, Validation Metric: 0.1950
Example output:
Epoch Number: 0, Batch Number: 11200, Training Loss: 16.9512
Time so far is 36m 58s
Epoch Number: 0, Batch Number: 11200, Validation Metric: 0.1700
Example output:
Epoch Number: 0, Batch Number: 11400, Training Loss: 17.3557
Time so far is 37m 39s
Epoch Number: 0, Batch Number: 11400, Validation Metric: 0.1550
Example output:
Epoch Number: 0,

Epoch Number: 0, Batch Number: 20600, Training Loss: 14.4312
Time so far is 69m 2s
Epoch Number: 0, Batch Number: 20600, Validation Metric: 0.3400
Example output:
Epoch Number: 0, Batch Number: 20800, Training Loss: 14.2262
Time so far is 69m 43s
Epoch Number: 0, Batch Number: 20800, Validation Metric: 0.2850
Example output:
Epoch Number: 0, Batch Number: 21000, Training Loss: 14.7246
Time so far is 70m 23s
Epoch Number: 0, Batch Number: 21000, Validation Metric: 0.2450
Example output:
Epoch Number: 0, Batch Number: 21200, Training Loss: 14.6068
Time so far is 71m 5s
Epoch Number: 0, Batch Number: 21200, Validation Metric: 0.2900
Example output:
Epoch Number: 0, Batch Number: 21400, Training Loss: 15.1851
Time so far is 71m 47s
Epoch Number: 0, Batch Number: 21400, Validation Metric: 0.2800
Example output:
Epoch Number: 0, Batch Number: 21600, Training Loss: 12.9268
Time so far is 72m 25s
Epoch Number: 0, Batch Number: 21600, Validation Metric: 0.3600
Example output:
Epoch Number: 0, B

Epoch Number: 0, Batch Number: 30600, Training Loss: 13.1161
Time so far is 103m 30s
Epoch Number: 0, Batch Number: 30600, Validation Metric: 0.3350
Example output:
Epoch Number: 0, Batch Number: 30800, Training Loss: 13.8373
Time so far is 104m 13s
Epoch Number: 0, Batch Number: 30800, Validation Metric: 0.3050
Example output:
Epoch Number: 0, Batch Number: 31000, Training Loss: 12.7411
Time so far is 104m 53s
Epoch Number: 0, Batch Number: 31000, Validation Metric: 0.3850
Example output:
Epoch Number: 0, Batch Number: 31200, Training Loss: 12.5863
Time so far is 105m 34s
Epoch Number: 0, Batch Number: 31200, Validation Metric: 0.3600
Example output:
Epoch Number: 0, Batch Number: 31400, Training Loss: 13.5674
Time so far is 106m 15s
Epoch Number: 0, Batch Number: 31400, Validation Metric: 0.3650
Example output:
Epoch Number: 0, Batch Number: 31600, Training Loss: 13.3088
Time so far is 106m 55s
Epoch Number: 0, Batch Number: 31600, Validation Metric: 0.3550
Example output:
Epoch Numb

Epoch Number: 0, Batch Number: 40600, Training Loss: 10.9017
Time so far is 137m 57s
Epoch Number: 0, Batch Number: 40600, Validation Metric: 0.4200
Example output:
Epoch Number: 0, Batch Number: 40800, Training Loss: 10.2842
Time so far is 138m 38s
Epoch Number: 0, Batch Number: 40800, Validation Metric: 0.3400
Example output:
Epoch Number: 0, Batch Number: 41000, Training Loss: 11.1045
Time so far is 139m 19s
Epoch Number: 0, Batch Number: 41000, Validation Metric: 0.3800
Example output:
Epoch Number: 0, Batch Number: 41200, Training Loss: 10.9204
Time so far is 140m 1s
Epoch Number: 0, Batch Number: 41200, Validation Metric: 0.3700
Example output:
Epoch Number: 0, Batch Number: 41400, Training Loss: 11.1925
Time so far is 140m 42s
Epoch Number: 0, Batch Number: 41400, Validation Metric: 0.3800
Example output:
Epoch Number: 0, Batch Number: 41600, Training Loss: 9.1726
Time so far is 141m 19s
Epoch Number: 0, Batch Number: 41600, Validation Metric: 0.4600
Example output:
Epoch Number

Epoch Number: 0, Batch Number: 50600, Training Loss: 10.6652
Time so far is 172m 31s
Epoch Number: 0, Batch Number: 50600, Validation Metric: 0.3400
Example output:
Epoch Number: 0, Batch Number: 50800, Training Loss: 10.2374
Time so far is 173m 14s
Epoch Number: 0, Batch Number: 50800, Validation Metric: 0.3400
Example output:
Epoch Number: 0, Batch Number: 51000, Training Loss: 9.7021
Time so far is 173m 54s
Epoch Number: 0, Batch Number: 51000, Validation Metric: 0.4000
Example output:
Epoch Number: 0, Batch Number: 51200, Training Loss: 8.7523
Time so far is 174m 35s
Epoch Number: 0, Batch Number: 51200, Validation Metric: 0.4200
Example output:
Epoch Number: 0, Batch Number: 51400, Training Loss: 10.3416
Time so far is 175m 17s
Epoch Number: 0, Batch Number: 51400, Validation Metric: 0.4000
Example output:
Epoch Number: 0, Batch Number: 51600, Training Loss: 9.5985
Time so far is 175m 57s
Epoch Number: 0, Batch Number: 51600, Validation Metric: 0.3950
Example output:
Epoch Number:

Epoch Number: 0, Batch Number: 60600, Training Loss: 7.7920
Time so far is 206m 58s
Epoch Number: 0, Batch Number: 60600, Validation Metric: 0.4100
Example output:
Epoch Number: 0, Batch Number: 60800, Training Loss: 9.1063
Time so far is 207m 40s
Epoch Number: 0, Batch Number: 60800, Validation Metric: 0.3850
Example output:
Epoch Number: 0, Batch Number: 61000, Training Loss: 8.8470
Time so far is 208m 21s
Epoch Number: 0, Batch Number: 61000, Validation Metric: 0.3800
Example output:
Epoch Number: 0, Batch Number: 61200, Training Loss: 8.4025
Time so far is 209m 3s
Epoch Number: 0, Batch Number: 61200, Validation Metric: 0.4100
Example output:
Epoch Number: 0, Batch Number: 61400, Training Loss: 7.3909
Time so far is 209m 42s
Epoch Number: 0, Batch Number: 61400, Validation Metric: 0.4000
Example output:
Epoch Number: 0, Batch Number: 61600, Training Loss: 9.9982
Time so far is 210m 24s
Epoch Number: 0, Batch Number: 61600, Validation Metric: 0.3500
Example output:
Epoch Number: 0, 